In [16]:
import cv2
import numpy as np
import pandas as pd
import os

region_data = pd.read_csv('regions.csv')
region_data

,cctv_folder_parent,map_image_name
0,SIMPANG 3 DPR,SIMPANG 3 DPR.jpeg
1,SIMPANG 4 CUNGKING,SIMPANG 4 CUNGKING.jpeg
2,SIMPANG 4 PATUNG KUDA,SIMPANG 4 PATUNG KUDA.jpeg
3,SIMPANG 5 PUSAT KOTA,SIMPANG 5 BANYUWANGI.jpeg


In [7]:
cctv_data = pd.read_csv('cctv_views.csv')
cctv_data

,region_name,cctv_view_name
0,SIMPANG 3 DPR,dpr selatan
1,SIMPANG 3 DPR,dpr timur
2,SIMPANG 3 DPR,dpr utara
3,SIMPANG 4 PATUNG KUDA,patungkuda barat
4,SIMPANG 4 PATUNG KUDA,patungkuda selatan
5,SIMPANG 4 PATUNG KUDA,patungkuda utara


In [8]:
# Define a CCTV path of image whose region has been detected
# Define the map path as well.

# You can freely change this depending on the condition of your database.
region_name = "SIMPANG 4 PATUNG KUDA"
cctv_range = {'start':0, 'end': 280}
cctv_time = '1200'

region_cctv_views = cctv_data[cctv_data['region_name'] == region_name]['cctv_view_name'].values.tolist()

region_map_path = region_data[region_data['cctv_folder_parent'] == region_name]
region_map_path = 'map/' + region_map_path['map_image_name'].values.tolist()[0]

region_coords = pd.read_csv('coords/cctv/{:}.csv'.format(region_name))

print(region_cctv_views)
print(region_map_path)
region_coords

['patungkuda barat', 'patungkuda selatan', 'patungkuda utara']
map/SIMPANG 4 PATUNG KUDA.jpeg


,view,pt1a,pt1b,pt1c,pt1d,pt2a,pt2b,pt2c,pt2d,pt3a,pt3b,pt3c,pt3d
0,patungkuda selatan,97 273,363 236,1074 476,187 770,0 0,1080 0,1080 720,0 720,400 717,295 644,392 513,492 587
1,patungkuda barat,132 283,396 262,1080 505,211 720,0 0,1080 0,1080 720,0 720,364 296,475 236,578 379,458 450
2,patungkuda utara,510 170,668 168,664 437,0 390,0 0,1080 0,1080 720,0 720,648 167,748 239,598 437,497 362


In [9]:
map_image = cv2.imread(region_map_path)

In [100]:
def get_points(pandas_frame, view_name):
    polygons = {}
    row = pandas_frame[pandas_frame['view'] == view_name]
    point_id = ['a', 'b', 'c', 'd']
    polygon_id = ['1', '2', '3']
    for i in polygon_id:
        polygon = []
        for j in point_id:
            point_name = 'pt'+ i + j
            point_array = [int(p) for p in row[point_name].values.tolist()[0].split()]
            polygon.append(point_array)
        polygons[i] = np.float32(polygon)
    return polygons

In [102]:
def warpPerspectivePoints(pts, M):
    trans = np.hstack((pts, np.ones((4,1)))) # Add ones at the bottom of matrix
    trans = np.matmul(M, np.transpose(trans))
    return np.transpose(trans[0:2,:] / trans[2:])

In [139]:
# Try for single video frames first
rctv = region_cctv_views[0]
path_cctv_view = './frame/cctv_{:}_{:} {:}_x264.mp4/'.format(region_name, rctv, cctv_time)
polygons = get_points(region_coords, rctv)

pts1 = polygons['1']
pts2 = polygons['3']

print("VIEW:", rctv)
print("PTS 1:", str(pts1).replace("\n", ""))
print("PTS 2:", str(pts2).replace("\n", ""))

# Get transformation matrix
M = cv2.getPerspectiveTransform(pts1,pts2)
pts_transform = np.int32(warpPerspectivePoints(pts1, M))

#cv2.drawContours(map_image, pts2, 0, (0,0,0), 2)

# Get file name length (it is assumed to be same for all the video frames)
file_ext_first_file = os.path.splitext(os.listdir(path_cctv_view)[0])
print("FIRST FILE:", file_ext_first_file)
filename_length = len(file_ext_first_file[0])

coco_vehicle_labels = ['bicycle', 'car', 'motorbike', 'bus', 'truck']

def strip(text):
    try:
        return text.strip()
    except AttributeError:
        return text
    
# Try for single frame video first
# Read CSV detection sequentially
for i in range(cctv_range['start'], cctv_range['end'] + 1):
    csv_file_name = str(i).zfill(filename_length)
    csv_path = '{:}/{:}.csv'.format(path_cctv_view, csv_file_name)
    
    # Read CSV
    objects = pd.read_csv(csv_path, names=["object", "probability", "left", "top", "right", "bottom"])
    print()
    print("OBJECTS:")
    print(objects)
    print()
    
    # Eliminate non-vehicles object
    objects = objects[objects['object'].isin(coco_vehicle_labels)]
    
    # Change string columns into numeric
    # -- Yeah, a little mistake I did when outputting CSV in darknet :( 
    cols = ['left', 'right', 'top', 'bottom']    
    for j in cols:
        objects[j] = pd.to_numeric(objects[j])
    
    # For every bounding box vehicles, find the bounding box center point    
    centers = objects.apply(lambda r: [
                      (r['left'] + r['right'])/2,
                      (r['top'] + r['bottom'])/2
                      ], axis=1)
    print("CENTERS:", str(centers).replace("\n", "").replace("  ", " "))
    
    # Transformation the points
    points_np = np.array([i for i in points])
    points_transform = np.int32(warpPerspectivePoints(points_np, M))
    print("CENTER TRANSFORMED", str(points_transform).replace("\n", ""))
    
    # Map the points to the image
    for i in points_transform:
        cv2.circle(map_image, tuple(i), 9, (0,255,0), -1)
        
    cv2.imshow('Image', map_image)
    while True:
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()
    break

VIEW: patungkuda barat
PTS 1: [[ 132.  283.] [ 396.  262.] [1080.  505.] [ 211.  720.]]
PTS 2: [[364. 296.] [475. 236.] [578. 379.] [458. 450.]]


error: OpenCV(3.4.1) C:\Miniconda3\conda-bld\opencv-suite_1533128839831\work\modules\imgproc\src\drawing.cpp:2515: error: (-215) npoints > 0 in function cv::drawContours


In [ ]:
### DO NOT RUN THIS

import numpy as np
import cv2

# ============================================================================

CANVAS_SIZE = (600,800)

FINAL_LINE_COLOR = (255, 255, 255)
WORKING_LINE_COLOR = (127, 127, 127)

# ============================================================================

class PolygonDrawer(object):
    def __init__(self, window_name):
        self.window_name = window_name # Name for our window

        self.done = False # Flag signalling we're done
        self.current = (0, 0) # Current position, so we can draw the line-in-progress
        self.points = [] # List of points defining our polygon


    def on_mouse(self, event, x, y, buttons, user_param):
        # Mouse callback that gets called for every mouse event (i.e. moving, clicking, etc.)

        if self.done: # Nothing more to do
            return

        if event == cv2.EVENT_MOUSEMOVE:
            # We want to be able to draw the line-in-progress, so update current mouse position
            self.current = (x, y)
        elif event == cv2.EVENT_LBUTTONDOWN:
            # Left click means adding a point at current position to the list of points
            print("Adding point #%d with position(%d,%d)" % (len(self.points), x, y))
            self.points.append((x, y))
        elif event == cv2.EVENT_RBUTTONDOWN:
            # Right click means we're done
            print("Completing polygon with %d points." % len(self.points))
            self.done = True


    def run(self):
        # Let's create our working window and set a mouse callback to handle events
        cv2.namedWindow(self.window_name, flags=cv2.CV_WINDOW_AUTOSIZE)
        cv2.imshow(self.window_name, np.zeros(CANVAS_SIZE, np.uint8))
        cv2.waitKey(1)
        cv2.cv.SetMouseCallback(self.window_name, self.on_mouse)

        while(not self.done):
            # This is our drawing loop, we just continuously draw new images
            # and show them in the named window
            canvas = np.zeros(CANVAS_SIZE, np.uint8)
            if (len(self.points) > 0):
                # Draw all the current polygon segments
                cv2.polylines(canvas, np.array([self.points]), False, FINAL_LINE_COLOR, 1)
                # And  also show what the current segment would look like
                cv2.line(canvas, self.points[-1], self.current, WORKING_LINE_COLOR)
            # Update the window
            cv2.imshow(self.window_name, canvas)
            # And wait 50ms before next iteration (this will pump window messages meanwhile)
            if cv2.waitKey(50) == 27: # ESC hit
                self.done = True

        # User finised entering the polygon points, so let's make the final drawing
        canvas = np.zeros(CANVAS_SIZE, np.uint8)
        # of a filled polygon
        if (len(self.points) > 0):
            cv2.fillPoly(canvas, np.array([self.points]), FINAL_LINE_COLOR)
        # And show it
        cv2.imshow(self.window_name, canvas)
        # Waiting for the user to press any key
        cv2.waitKey()

        cv2.destroyWindow(self.window_name)
        return canvas

# ============================================================================

if __name__ == "__main__":
    pd = PolygonDrawer("Polygon")
    image = pd.run()
    cv2.imwrite("polygon.png", image)
    print("Polygon = %s" % pd.points)

In [140]:

    cv2.destroyAllWindows()

IndentationError: unexpected indent (<ipython-input-140-4f77a75c7d83>, line 2)

In [ ]:

# # Eliminate non-vehicles point (can be obtained from the "object" header)
  # For example, vehicles: truck, car, motorbike
# # For every bounding box vehicles:
# # # Find the bounding box center point
# # # Map the point according to transformation matrix M
# # # Done.
# # Now, for displaying purposes:
# # Draw the map and all the transformed points
# # In new window, draw the CCTV image with boxes

parent_folder = kuda_cctv_path
image_map = kuda_cctv_map
 
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture(parent_folder + '\patungkuda selatan 1200_x264.mp4')
map = cv2.imread(image_map)
 
# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

x = 0
dim_x, dim_y = None, None

pts1 = np.float32([[97, 273],[363, 236], [1074, 476], [187, 770]])
pts2 = np.float32([[0, 0], [1080, 0], [1080, 720], [0, 720]])/2

M = cv2.getPerspectiveTransform(pts1,pts2)
pts_transform = np.int32(warpPerspectivePoints(pts1, M))

ii = 0
ord_q = ord('q') # For quit button
ord_p = ord('p') # For pause button

dim_x = np.max(pts2[:,0])
dim_y = np.max(pts2[:,1])

finished = False
try:
    while(cap.isOpened() and not finished):
        ret, frame = cap.read()
        if ret == True:

            f = frame
            dst = cv2.warpPerspective(f,M,(dim_x, dim_y))
            
            for i in pts_transform:
                cv2.circle(dst, tuple(i), 9, (0,255,0), -1)
            
            for i in pts1:
                cv2.circle(f, tuple(i), 9, (0,255,0), -1)
                
            cv2.imshow('Frame2',f)
            cv2.imshow('Frame', dst)
            
            # Press Q on keyboard to  exit
k = cv2.waitKey(25) & 0xFF
cv2.destroyAllWindows()

In [ ]:
# # Eliminate non-vehicles point (can be obtained from the "object" header)
  # For example, vehicles: truck, car, motorbike
# # For every bounding box vehicles:
# # # Find the bounding box center point
# # # Map the point according to transformation matrix M
# # # Done.
# # Now, for displaying purposes:
# # Draw the map and all the transformed points
# # In new window, draw the CCTV image with boxes

In [ ]:
# Get the coordinates and matrix coordinates